## Rename original files for easy parsing (Drop DLC part from the file names)

In [20]:
import os
import re

folder = r"C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Analyzed-videos-filtered"

# Define known multi-word city names to fix
city_names = [
    'El_Paso', 'New_York', 'New_Jersey', 'New_Orleans',
    'Las_Cruces', 'Ft_Worth'
]
city_replacements = {name: name.replace('_', '') for name in city_names}

for file in os.listdir(folder):
    if not file.endswith(".csv"):
        continue

    original_path = os.path.join(folder, file)
    new_name = file

    # Step 1: Remove DLC suffix (if it still exists)
    new_name = re.sub(r'DLC_resnet50_.*?_filtered', '', new_name)

    # Step 2: Replace 'None_#' with 'None#'
    new_name = re.sub(r'None_(\d+)', r'None\1', new_name)

    # Step 3: Replace multi-word city names
    for old, new in city_replacements.items():
        new_name = new_name.replace(old, new)

    # Skip if no changes needed
    if new_name == file:
        continue

    new_path = os.path.join(folder, new_name)

    # Safety: skip if destination file already exists
    if os.path.exists(new_path):
        print(f"❗Skipped (target exists): {new_name}")
        continue

    # Rename file
    os.rename(original_path, new_path)
    print(f"✅ Renamed: {file}  -->  {new_name}")


✅ Renamed: Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1_Houston_Austin_Toronto_Berlin_Trial_1_bottom_leftDLC_resnet50_DLCDec25shuffle1_1140000_filtered.csv  -->  Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1_Houston_Austin_Toronto_Berlin_Trial_1_bottom_left.csv
✅ Renamed: Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1_Houston_Austin_Toronto_Berlin_Trial_1_bottom_rightDLC_resnet50_DLCDec25shuffle1_1140000_filtered.csv  -->  Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1_Houston_Austin_Toronto_Berlin_Trial_1_bottom_right.csv
✅ Renamed: Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1_Houston_Austin_Toronto_Berlin_Trial_1_top_leftDLC_resnet50_DLCDec25shuffle1_1140000_filtered.csv  -->  Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1_Houston_Austin_Toronto_Berlin_Trial_1_top_left.csv
✅ Renamed: Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1_Houston_Austin_Toronto_Berlin_Trial_1_top_rightDLC_resnet50_DLCDec25shuffle1_1140000_filtered.csv  -->  Cre_and_WT_Habituation_Food_and_M

## Separate name block with '____'

In [21]:
import os
import re

folder = r"C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Analyzed-videos-filtered"

for file in os.listdir(folder):
    if not file.endswith(".csv"):
        continue

    if "____Trial" in file:
        continue  # Already processed

    # Match the position of Trial_#
    trial_match = re.search(r'_Trial_\d+', file)
    if not trial_match:
        print(f"❌ Skipped (no Trial found): {file}")
        continue

    trial_start = trial_match.start()
    before_trial = file[:trial_start]
    after_trial = file[trial_start:]  # Includes _Trial_1...

    tokens = before_trial.split('_')
    if len(tokens) < 4:
        print(f"❌ Skipped (not enough tokens before Trial): {file}")
        continue

    # Get last 4 tokens = animal block
    animal_tokens = tokens[-4:]
    animal_block = '_'.join(animal_tokens)

    # Trim trailing underscore before animal block (if any)
    prefix = '_'.join(tokens[:-4])
    if prefix.endswith('_'):
        prefix = prefix[:-1]  # Remove trailing underscore

    # Rebuild filename
    new_file = f"{prefix}____{animal_block}____{after_trial}"

    original_path = os.path.join(folder, file)
    new_path = os.path.join(folder, new_file)

    if os.path.exists(new_path):
        print(f"⚠️ Skipped (target exists): {new_file}")
        continue

    os.rename(original_path, new_path)
    print(f"✅ Renamed:\n{file}  -->\n{new_file}\n")


✅ Renamed:
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1_Houston_Austin_Toronto_Berlin_Trial_1_bottom_left.csv  -->
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_bottom_left.csv

✅ Renamed:
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1_Houston_Austin_Toronto_Berlin_Trial_1_bottom_right.csv  -->
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_bottom_right.csv

✅ Renamed:
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1_Houston_Austin_Toronto_Berlin_Trial_1_top_left.csv  -->
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_top_left.csv

✅ Renamed:
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1_Houston_Austin_Toronto_Berlin_Trial_1_top_right.csv  -->
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_top_right.csv

✅ Renamed:
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2_Denver_ElPaso_Waco_FtWorth_T

## Extract animal name for the file name

In [2]:
import re

def extract_animal_name(filename):
    # Mapping from quadrant to animal position
    quadrant_to_position = {
        'top_left': 0,
        'top_right': 1,
        'bottom_left': 2,
        'bottom_right': 3,
    }

    # Extract quadrant
    match_quadrant = re.search(r'(top_left|top_right|bottom_left|bottom_right)', filename)
    if not match_quadrant:
        return None

    quadrant = match_quadrant.group(1)
    animal_index = quadrant_to_position[quadrant]

    # Match pattern between two underscore groups of 4 or more before Trial
    match_animal_block = re.search(r'_{4,}(.*?)_{4,}Trial_\d+', filename)
    if not match_animal_block:
        return None

    animal_block = match_animal_block.group(1)
    animal_list = animal_block.split('_')

    if animal_index >= len(animal_list):
        return None

    return animal_list[animal_index]


In [9]:
filename1 = 'Toy_Light_CRE_WT_Gen_2_Repeat_WT_Y_CNO_2_24_25_S4_Y_CNO____Paris_London_Phoenix_None4_____Trial_1_top_right.csv'
filename2 = 'Toy_Light_CRE_WT_Gen_2_3rd_Attempt_2_5_25_S4_P_CNO____M_P_Q_None4_____Trial_1_bottom_right.csv'

print(extract_animal_name(filename1))  # Output: London
print(extract_animal_name(filename2))  # Output: K

London
None4


## Validation

In [13]:
import os

# Folder path
folder = r"C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Analyzed-videos-filtered"

for file in os.listdir(folder):
    if file.endswith(".csv"):
        animal_name = extract_animal_name(file)
        if animal_name is not None:
            print(f"{file} --> {animal_name}")
        else:
            print(f"[SKIPPED] Pattern not matched: {file}")


Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_bottom_left.csv --> Toronto
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_bottom_right.csv --> Berlin
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_top_left.csv --> Houston
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_top_right.csv --> Austin
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_bottom_left.csv --> Waco
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_bottom_right.csv --> FtWorth
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_top_left.csv --> Denver
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_top_right.csv --> ElPaso
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S3___